1.Install Dependencies For Running

In [ ]:
!apt update
!apt install -y g++

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
51 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as

Here is the main cpp code for merge sort acceleration

In [ ]:
%%writefile mergesort_parallel.cpp
#include <stdio.h>
#include <stdlib.h>
#include <errno.h>
#include <sys/time.h>

#include <iostream>
#include <algorithm>
// Add this head file OpenMP library for parallelism
#include <omp.h>

/**
 * Helper routine: check if array is sorted correctly by parallel
 */
bool isSorted(int ref[], int data[], const size_t size)
{
	std::sort(ref, ref + size);
	for (size_t idx = 0; idx < size; ++idx)
	{
		if (ref[idx] != data[idx])
		{
			return false;
		}
	}
	return true;
}

/**
 * sequential merge step without parallel by halves, just copy from original code
 */
void MsMergeSequential(int *out, int *in, long begin1, long end1, long begin2, long end2, long outBegin)
{
	long left = begin1;
	long right = begin2;

	long idx = outBegin;

	while (left < end1 && right < end2)
	{
		if (in[left] <= in[right])
		{
			out[idx] = in[left];
			left++;
		}
		else
		{
			out[idx] = in[right];
			right++;
		}
		idx++;
	}

	while (left < end1)
	{
		out[idx] = in[left];
		left++, idx++;
	}

	while (right < end2)
	{
		out[idx] = in[right];
		right++, idx++;
	}
}

/**
 * 1.Parallel merge algorithm by using double tasks for each merge step and moving towards the middle separately
 * 2.There is also a cutOffMerge threshold value to control if using the parallel version. When the sub array
 * is big enough we can use parallel merge for acceleration. If too small, parallelization by OpenMp tasks will
 * in turn increase the total overhead for managing.
 */
void MsMergeParallel(int *out, int *in, long begin1, long end1, long begin2, long end2, long outBegin)
{
	long left = begin1;
	long right = end2 - 1;
	long outLeft = outBegin;
	long outRight = outBegin + (end1 - begin1) + (end2 - begin2) - 1;
	long mid = (outLeft + outRight) / 2; // Define the middle point for stopping

#pragma omp parallel
	{
#pragma omp single // use single thread multiple tasks
		{
// Task 1: merge arrays from left (smallest value) to middle point.
#pragma omp task shared(out, in)
			{
				long l = left;
				long r = begin2;
				long o = outLeft;
				while (l < end1 && r < end2 && o <= mid)
				{ // add another condition that stopping at the middle point when moving right
					if (in[l] <= in[r])
					{
						out[o++] = in[l++];
					}
					else
					{
						out[o++] = in[r++];
					}
				}
				// merge the remaining part elements in case one sub array finished all
				while (l < end1 && o <= mid)
				{
					out[o++] = in[l++];
				}
				while (r < end2 && o <= mid)
				{
					out[o++] = in[r++];
				}
			}

// Task 2: merge arrays from right (largest value) to middle point.
#pragma omp task shared(out, in)
			{
				long l = end1 - 1;
				long r = end2 - 1;
				long o = outRight;
				while (l >= begin1 && r >= begin2 && o > mid)
				{ // add another condition that stopping at the middle point when moving left
					if (in[l] >= in[r])
					{
						out[o--] = in[l--];
					}
					else
					{
						out[o--] = in[r--];
					}
				}
				// merge the remaining part elements in case one sub array finished all
				while (l >= begin1 && o > mid)
				{
					out[o--] = in[l--];
				}
				while (r >= begin2 && o > mid)
				{
					out[o--] = in[r--];
				}
			}

#pragma omp taskwait // Wait to synchronize double tasks
		}
	}
}

/**
 * Parallel MergeSort with parallel by multiple tasks and cut-off standard
 */
void MsSequential(int *array, int *tmp, bool inplace, long begin, long end, int depth, long cutOffSort, long cutOffMerge)
{
	if ((end - begin) <= cutOffSort)
	{
		// If below cut-off, perform a default sequential sort instead
		if (inplace)
		{
			std::sort(array + begin, array + end);
		}
		else
		{
			std::copy(array + begin, array + end, tmp + begin);
			std::sort(tmp + begin, tmp + end);
		}
		return;
	}

	const long half = (begin + end) / 2;

	// Parallel the recursive calls by shared tasks when depth allows parallelism
	if (depth > 0)
	{
#pragma omp task shared(array, tmp)
		MsSequential(array, tmp, !inplace, begin, half, depth - 1, cutOffSort, cutOffMerge); // depth - 1

#pragma omp task shared(array, tmp)
		MsSequential(array, tmp, !inplace, half, end, depth - 1, cutOffSort, cutOffMerge); // depth - 1

#pragma omp taskwait // Wait to synchronize tasks
	}
	else
	{
		// Do original sequential algorithm once reaching the depth limitation
		// its value will be unchanged anymore, which means alway zero
		MsSequential(array, tmp, !inplace, begin, half, depth, cutOffSort, cutOffMerge);
		MsSequential(array, tmp, !inplace, half, end, depth, cutOffSort, cutOffMerge);
	}

	// Merge the results according the cutOffMerge threshold value
	bool mergeByParallel = (end - begin) >= cutOffMerge;
	if (mergeByParallel)
	{
		// Parallel: if the arrays to merge is large enough which means we can benefit from parallel
		if (inplace)
		{
			MsMergeParallel(array, tmp, begin, half, half, end, begin);
		}
		else
		{
			MsMergeParallel(tmp, array, begin, half, half, end, begin);
		}
	}
	else
	{
		// Sequential: if the arrays are too small to parallel them (overload for too many tasks to manage)
		if (inplace)
		{
			MsMergeSequential(array, tmp, begin, half, half, end, begin);
		}
		else
		{
			MsMergeSequential(tmp, array, begin, half, half, end, begin);
		}
	}
}

/**
 * Parallel MergeSort - creates the parallel region here
 */
void MsSerial(int *array, int *tmp, const size_t size, int maxDepth, long cutOffSort, long cutOffMerge)
{
// Create pool of threads and start with one of them.
#pragma omp parallel // parallel region
	{
#pragma omp single // Ensure only one thread starts tasks
		{
			MsSequential(array, tmp, true, 0, size, maxDepth, cutOffSort, cutOffMerge);
		}
	}
}

/**
 * Program entry point
 */
int main(int argc, char *argv[])
{
	// Variables to measure the elapsed time
	struct timeval t1, t2;
	double etime;

	// Expect one command line argument: array size
	if (argc != 2)
	{
		printf("Usage: MergeSort.exe <array size> \n");
		printf("\n");
		return EXIT_FAILURE;
	}
	else
	{
		const size_t stSize = strtol(argv[1], NULL, 10);
		int *data = (int *)malloc(stSize * sizeof(int));
		int *tmp = (int *)malloc(stSize * sizeof(int));
		int *ref = (int *)malloc(stSize * sizeof(int));

		printf("Initialization...\n");

		srand(95);
		for (size_t idx = 0; idx < stSize; ++idx)
		{
			data[idx] = (int)(stSize * (double(rand()) / RAND_MAX));
		}
		std::copy(data, data + stSize, ref);

		double dSize = (stSize * sizeof(int)) / 1024 / 1024;
		printf("Sorting %zu elements of type int (%f MiB)...\n", stSize, dSize);

		// Start measuring time
		gettimeofday(&t1, NULL);

		// Example parameters: maxDepth = 4, cutOffSort = 16, cutoffMerge = 2048, input datasize = 1000000000
		// 1. If depth is too large, high level of parallel but also high tasks pressure, worse performance;
		// -  If depth is too low we haven't fully utilized the whole threads for acceleration.
		// 2. High cutOffSort, to big sub array to sort sequentially, low speed;
		// -  Low cutOffSort too many small arrays to deal with
		// 3. High cutOffMerge like 8192, arrays to merge at lower levels can't benefit from this parallelism;
		// -  Low cutOffSort, too many small paralleled tasks to manage which leading bad performance.
		const long depth = 4;
		const long cutOffSort = 16;
		const long cutOffMerge = 2048;
		// Perform the mergesort algorithm
		MsSerial(data, tmp, stSize, depth, cutOffSort, cutOffMerge);

		gettimeofday(&t2, NULL);

		// Calculate elapsed time
		etime = (t2.tv_sec - t1.tv_sec) * 1000 + (t2.tv_usec - t1.tv_usec) / 1000;
		etime = etime / 1000;

		printf("done, took %f sec. Verification...", etime);
		if (isSorted(ref, data, stSize))
		{
			printf(" successful.\n");
			printf("With parameter depth: %ld, cutOffSort: %ld, cutOffMerge: %ld\n", depth, cutOffSort, cutOffMerge);
		}
		else
		{
			printf(" FAILED.\n");
		}

		free(data);
		free(tmp);
		free(ref);
	}

	return EXIT_SUCCESS;
}

Overwriting mergesort_parallel.cpp


2. Nextly compile it with OpenMP

In [ ]:
!g++ -fopenmp -o mergesort_parallel mergesort_parallel.cpp


We can also check how many threads can we use at colab (actually just two)

In [ ]:
!nproc --all


2


3. Now run the exe with data size changing ascent

In [ ]:
!./mergesort_parallel 1000000  # With the array size = 10^6

Initialization...
Sorting 1000000 elements of type int (3.000000 MiB)...
done, took 0.165000 sec. Verification... successful.
With parameter depth: 4, cutOffSort: 16, cutOffMerge: 2048


In [ ]:
!./mergesort_parallel 10000000  # With the array size = 10^7

Initialization...
Sorting 10000000 elements of type int (38.000000 MiB)...
done, took 2.343000 sec. Verification... successful.
With parameter depth: 4, cutOffSort: 16, cutOffMerge: 2048


In [ ]:
!./mergesort_parallel 100000000  # With the array size = 10^8

Initialization...
Sorting 100000000 elements of type int (381.000000 MiB)...
done, took 24.344000 sec. Verification... successful.
With parameter depth: 4, cutOffSort: 16, cutOffMerge: 2048


In [ ]:
!./mergesort_parallel 1000000000  # With the array size = 10^9, too long to wait because of just 2 thread which can be used

Initialization...
Sorting 1000000000 elements of type int (3814.000000 MiB)...
^C
